# データと計量経済学 第2回 ベイズ推定の基本(その2)

## 参考文献
- 完全独習 ベイズ統計学入門 小島　寛之 著 ダイヤモンド社
    - https://www.diamond.co.jp/book/9784478013328.html

In [1]:
# ライブラリのimport

# NumPy用ライブラリ
import numpy as np
# Matplotlib中のpyplotライブラリのインポート
import matplotlib.pyplot as plt
# matplotlib日本語化対応ライブラリのインポート
import japanize_matplotlib



- **BayesianEstimationAreaChartAnalyzer**というクラスを用意しました。

In [54]:
class BayesianEstimationAreaChartAnalyzer:
    def __init__(self, conditional_probabilities, labels, prior_probabilities):
        """
        分析クラスを初期化し、データを保持します。

        Args:
            conditional_probabilities (list of lists): 条件付確率のリスト（形状は [カテゴリ][スタック]）。
            labels (list): 各カテゴリのラベル。
            prior_probabilities (list): 事前確率のリスト（割合指定、合計が1である必要あり）。
        """
        self.conditional_probabilities = np.array(conditional_probabilities)
        self.labels = labels
        self.prior_probabilities = prior_probabilities

    def plot_prior_probabilities_chart(self, colors=None, title="事前確率のみを表示する棒グラフ", xlabel="カテゴリ", ylabel="割合", text_offset_ratio=0.05):
        """
        事前確率のみを使用して棒グラフをプロットします。

        棒グラフは縦に分割されず、データラベルも表示しません。
        各カテゴリの上部には事前確率の値を表示します。
        """
        num_categories = len(self.prior_probabilities)

        if colors is None:
            colors = plt.cm.tab10.colors  # デフォルトの色パレット

        if not np.isclose(sum(self.prior_probabilities), 1.0):
            raise ValueError("prior_probabilitiesの合計は1である必要があります。")

        x_positions = np.cumsum([0] + self.prior_probabilities[:-1])  # 各カテゴリの左端位置
        y_max = 1.0  # グラフ全体の高さを1.0に設定
        fig, ax = plt.subplots(figsize=(8, 6))

        # 上部と右部の枠線を削除
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        # 各カテゴリのバーをプロット
        for i, (label, x_pos, width) in enumerate(zip(self.labels, x_positions, self.prior_probabilities)):
            bar = ax.bar(
                x_pos,
                y_max,
                width=width,
                bottom=0,
                color='white',
                edgecolor='black',
                align='edge',
                label=f"{label}"
            )

            # 事前確率をカテゴリごとの上部に表示
            text_y_position = y_max * (1 + text_offset_ratio)
            ax.text(
                x_pos + width / 2,
                text_y_position,
                f"{width:.2f}",
                ha='center',
                va='bottom',
                fontsize=10,
                color="black"
            )

        # 区切り線を描画
        for x_pos in x_positions:
            ax.axvline(x_pos, color="gray", linestyle="--", linewidth=1)
        ax.axvline(x_positions[-1] + self.prior_probabilities[-1], color="gray", linestyle="--", linewidth=1)

        ax.set_xticks(x_positions + np.array(self.prior_probabilities) / 2)
        ax.set_xticklabels(self.labels)
        ax.set_ylim(0, y_max * (1 + text_offset_ratio * 2))
        ax.set_title(title)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)

        plt.tight_layout()
        plt.show()

    def plot_combined_stacked_bar_chart(self, colors=None, title="データラベルを2行表示する積み上げ棒グラフ", xlabel="カテゴリ", ylabel="割合", text_offset_ratio=0.05):
        """
        一つのグラフで、条件付確率と事前確率×条件付確率の積の両方のデータラベルを表示します。

        各バーには以下の形式で2行のデータラベルが表示されます：
        条件付確率：数値
        全体の確率：数値
        """
        num_categories, num_stacks = self.conditional_probabilities.shape

        if colors is None:
            colors = plt.cm.tab10.colors  # デフォルトの色パレット

        if not np.isclose(sum(self.prior_probabilities), 1.0):
            raise ValueError("prior_probabilitiesの合計は1である必要があります。")

        x_positions = np.cumsum([0] + self.prior_probabilities[:-1])  # 各カテゴリの左端位置
        y_max = 1.0  # グラフ全体の最大値を1.0に設定
        fig, ax = plt.subplots(figsize=(8, 6))

        # 上部と右部の枠線を削除
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        for i, (category_data, label, x_pos, width) in enumerate(zip(self.conditional_probabilities, self.labels, x_positions, self.prior_probabilities)):
            # データの順序を逆にする
            category_data_reversed = category_data[::-1]

            # bottomsを再計算
            bottoms = np.cumsum(category_data_reversed) - category_data_reversed

            bars = ax.bar(
                x_pos,
                category_data_reversed,
                width=width,
                bottom=bottoms,
                facecolor='none',   # 塗りつぶしなし
                edgecolor='black',  # 枠線の色を黒色に指定
                align='edge',
                label=f"{label}"
            )

            # 各バーに2行のデータラベルを表示
            for bar, cond_prob in zip(bars, category_data_reversed):
                bar_center = bar.get_y() + bar.get_height() / 2  # バーの中央位置
                multiplied_value = cond_prob * width
                label_text = f"条件付確率：{cond_prob:.2f}\n全体の確率：{multiplied_value:.2f}"
                ax.text(
                    bar.get_x() + width / 2,
                    bar_center,
                    label_text,
                    ha='center',
                    va='center',
                    color="black",
                    fontsize=10
                )

            # 事前確率をカテゴリごとの上部に表示
            text_y_position = y_max * (1 + text_offset_ratio)
            ax.text(
                x_pos + width / 2,
                text_y_position,
                f"{width:.2f}",
                ha='center',
                va='bottom',
                fontsize=10,
                color="black"
            )

        # 区切り線を描画
        for x_pos in x_positions:
            ax.axvline(x_pos, color="gray", linestyle="--", linewidth=1)
        ax.axvline(x_positions[-1] + self.prior_probabilities[-1], color="gray", linestyle="--", linewidth=1)

        ax.set_xticks(x_positions + np.array(self.prior_probabilities) / 2)
        ax.set_xticklabels(self.labels)
        ax.set_ylim(0, y_max * (1 + text_offset_ratio * 2))
        ax.set_title(title)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)

        plt.tight_layout()
        plt.show()

    def plot_highlighted_stacked_bar_chart(self, index, colors=None, title="指定インデックスのスタックのみ表示", xlabel="カテゴリ", ylabel="割合", text_offset_ratio=0.05):
        """
        指定したインデックス以外のスタックを灰色で表示し、指定したスタックのみ強調表示するグラフをプロットします。

        Args:
            index (int): 強調表示したいスタックのインデックス（0始まり）。
        """
        import matplotlib.pyplot as plt
        import numpy as np

        num_categories, num_stacks = self.conditional_probabilities.shape

        if colors is None:
            colors = plt.cm.tab10.colors

        if not np.isclose(sum(self.prior_probabilities), 1.0):
            raise ValueError("prior_probabilitiesの合計は1である必要があります。")

        x_positions = np.cumsum([0] + self.prior_probabilities[:-1])
        y_max = 1.0  # グラフ全体の最大値を1.0に設定
        fig, ax = plt.subplots(figsize=(8, 6))

        # 上部と右部の枠線を削除
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        for i, (category_data, label, x_pos, width) in enumerate(zip(self.conditional_probabilities, self.labels, x_positions, self.prior_probabilities)):
            # データの順序を逆にする
            category_data_reversed = category_data[::-1]

            # bottomsを再計算
            bottoms = np.cumsum(category_data_reversed) - category_data_reversed

            # インデックスも逆順に対応させる
            reversed_index = num_stacks - 1 - index

            for j in range(num_stacks):
                if j == reversed_index:
                    facecolor = 'white'
                    edgecolor = 'black'
                    show_label = True
                else:
                    facecolor = 'gray'
                    edgecolor = 'black'
                    show_label = False

                bar = ax.bar(
                    x_pos,
                    category_data_reversed[j],
                    width=width,
                    bottom=bottoms[j],
                    facecolor=facecolor,
                    edgecolor=edgecolor,
                    align='edge'
                )

                multiplied_value = category_data_reversed[j] * width

                if show_label and multiplied_value > 0:
                    bar_center = bottoms[j] + category_data_reversed[j] / 2
                    ax.text(
                        x_pos + width / 2,
                        bar_center,
                        f"{multiplied_value:.2f}",
                        ha='center',
                        va='center',
                        color="black",
                        fontsize=10
                    )

            # 事前確率をカテゴリごとの上部に表示
            text_y_position = y_max * (1 + text_offset_ratio)
            ax.text(
                x_pos + width / 2,
                text_y_position,
                f"{width:.2f}",
                ha='center',
                va='bottom',
                fontsize=10,
                color="black"
            )

        # 区切り線を描画
        for x_pos in x_positions:
            ax.axvline(x_pos, color="gray", linestyle="--", linewidth=1)
        ax.axvline(x_positions[-1] + self.prior_probabilities[-1], color="gray", linestyle="--", linewidth=1)

        ax.set_xticks(x_positions + np.array(self.prior_probabilities) / 2)
        ax.set_xticklabels(self.labels)
        ax.set_ylim(0, y_max * (1 + text_offset_ratio * 2))
        ax.set_title(title)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)

        plt.tight_layout()
        plt.show()

        # 事後確率を計算
        numerator = self.prior_probabilities * self.conditional_probabilities[:, index]
        denominator = np.sum(numerator)
        posterior_probs = numerator / denominator

        # ラベルと事後確率を表示
        print("事後確率:")
        for label, prob in zip(self.labels, posterior_probs):
            print(f"{label}: {prob:.2%}")

    def calculate_posterior_probabilities(self, index):
        """
        指定されたスタックのインデックスに基づいて事後確率を計算し、リストで返します。

        Args:
            index (int): 事後確率を計算するためのスタックのインデックス（0始まり）。

        Returns:
            list: 各カテゴリに対応する事後確率のリスト（合計は1）。
        """
        num_stacks = self.conditional_probabilities.shape[1]
        if index < 0 or index >= num_stacks:
            raise ValueError(f"indexは0から{num_stacks - 1}の範囲で指定してください。")

        # 各カテゴリごとに、事前確率と条件付確率の積を計算
        multiplied_values = []
        for prior, cond_probs in zip(self.prior_probabilities, self.conditional_probabilities):
            cond_prob = cond_probs[index]
            multiplied_value = prior * cond_prob
            multiplied_values.append(multiplied_value)

        # 合計を計算して正規化
        total = sum(multiplied_values)
        if total == 0:
            raise ValueError("合計が0になるため、事後確率を計算できません。")

        posterior_probabilities = [value / total for value in multiplied_values]

        return posterior_probabilities

## 1.おさらい
- ベイズ推定の手続きの基本は以下という話でした。
    - (事前確率) -> (条件付き確率) -> (観測による情報の入手) -> (事後確率)


- 用語確認
    - 事前確率(prior probability)
    - 条件付き確率(conditional probability)
    - 事後確率(posterior probability)


- 前回の「買う人」「ひやかし」の例で確認しましょう。


- 事前確率：

| 買う人 | ひやかし |
| ---- | ---- |
|0.2 | 0.8|

- 条件付き確率：

| タイプ | 声かけの確率 | 声かけしない確率 |
| ---- | ---- | ---- |
|買う人 | 0.9 | 0.1|
|ひやかし | 0.3 | 0.7 |

- 実行例です。

In [ ]:
# サンプルデータの定義
prior_probabilities = [0.2, 0.8]
labels = ['買う人', 'ひやかし']

conditional_probabilities = [
    [0.9, 0.1],
    [0.3, 0.7]
]

# クラスのインスタンス化
analyzer = BayesianEstimationAreaChartAnalyzer(conditional_probabilities, labels, prior_probabilities)

# 1. 事前確率のみを表示するグラフをプロット
analyzer.plot_prior_probabilities_chart(title="事前確率")

# 2. データラベルを2行表示する積み上げ棒グラフをプロット
analyzer.plot_combined_stacked_bar_chart(title="条件付き確率")

# 3. 特定のスタックを強調表示するグラフをプロット
## 次の4も中で呼び出している。
index = 0
analyzer.plot_highlighted_stacked_bar_chart(index, title=f"インデックス{index}を観測した場合")

# 4. 事後確率を計算して返す(indexを指定)
posterior_probs = analyzer.calculate_posterior_probabilities(index)
print("4.数値のリストだけを計算して入手して表示")
print(posterior_probs)

- 上記で分かる通り、事後確率も計算できるようにしました。
- 上の例は、声かけがあった場合でした。声かけがなかった場合も実行してみましょう。

In [ ]:
# 3. 特定のスタックを強調表示するグラフをプロット
## 次の4も中で呼び出している。
index = 1
analyzer.plot_highlighted_stacked_bar_chart(index, title=f"インデックス{index}を観測した場合")

----
### 問1 ベイズ推定ができるクラスを具体的な問題に適用してみよう。
----

 #### 問1-1 スパムメールの分類(カテゴリ2×2)
- 背景:
> 受信したメールが「スパム」か「通常メール」かを分類したい。メールがスパムかどうかは直接には分かりませんが、特定のキーワード（例えば「無料」）が含まれているかどうかが分かります。

- 事前確率:
    - スパムメールの割合:20%
    - 通常メールの割合:80%

- 条件付き確率:
    - スパムメールがキーワードを含む確率:70%
    - 通常メールがキーワードを含む確率:10%
- 観測:
    - メールに特定のキーワードが含まれている。
- 目的:
    - そのメールがスパムである事後確率を求める。

In [ ]:
# ここに書く

#### 問1-2 商品の不良率と生産ライン(カテゴリ3×2)
- 背景:
> 工場には3つの生産ラインがあり、それぞれライン1、ライン2、ライン3とします。製品がどの生産ラインから出荷されたかはわかりませんが、その製品が不良品かどうかを検査することができます。

- 事前確率:
    - ライン1から出荷される確率: 40%
    - ライン2から出荷される確率: 35%
    - ライン3から出荷される確率: 25%
- 条件付き確率:
    - 各生産ラインでの不良品率（製品が不良品である確率）。
        - ライン1の場合: 5%
        - ライン2の場合: 10%
        - ライン3の場合: 15%
- 観測:
    - 製品が不良品であった。

- 目的:
    - この観測に基づいて、製品が各生産ラインから出荷された事後確率を求める。

In [59]:
# ここに書く

#### 問1-3 試験の評価と学生の熱心さ(カテゴリ2×3)
- 背景:
> 学生はある試験を受け、その結果は「不合格」、「合格」、「優秀」の3段階で評価されます。我々は学生が「勉強熱心」か「勉強熱心でない」かの2つのカテゴリに分類されるとします。しかし、直接にはどちらのカテゴリか分かりません。

- 事前確率:
    - 勉強熱心な学生の割合: 40%
    - 勉強熱心でない学生の割合: 60%
- 条件付き確率:
    - 各カテゴリの学生が試験で以下の評価を受ける確率:
        - 勉強熱心な学生:
            - 不合格: 10%
            - 合格: 60%
            - 優秀: 30%
        - 勉強熱心でない学生:
            - 不合格: 50%
            - 合格: 40%
            - 優秀: 10%
- 観測:
    - ある学生が試験で「優秀」の評価を得た。
- 目的:
    - この観測に基づいて、その学生が勉強熱心である事後確率を求める。

In [60]:
# ここに書く

## 2.事前確率が未知の場合(主観確率)

参考図書P.44




### 2.1 問題設定：チョコをくれた彼女の気持ちを推定する

- 以下の状況を考えます。
> あなたが男性であると仮定し、特定の同僚女性さんが自分に好意を持っているかどうか気になっているとする。そんな中、あなたはバレンタインデーに彼女からチョコレートを貰った。さて、あなたは、彼女が自分を本命と考えている確率をいくつと推定すべきか。





### 2.2 ステップ1：事前確率の設定(しかし事前確率は未知である)

- ここでのポイントは、「彼女があなたをどの程度、本命と思っているか」という人の心の中を(事前確率として)数値化しなくてはならないということです。
    - 仮に「その同僚女性(同じ人)がたくさんいたら、そのうちの何%があなたを本命と思うか」という問いであれば「確率」の問題と考えることができますが、同僚女性(同じ人)は一人しかいないので、そんな問題は成り立ちません。
    - こうしたことを扱えるのがベイズ推定の強みとも言えます。


- あなたのことを好きなことを「**本命**」、そうでないことを「**論外**」と呼ぶことにします。
- 単に全く分からないので、「**とりあえず同じ確率にする**」ということで「**0.5**」と「**0.5**」にします。
    - このように、分からないのでとりあえず同じ確率であると考えることを「**理由不十分の原理**」と呼びます。

- 表にするとこうです。

- 事前確率：

| 本命 | 論外 |
| ---- | ---- |
|0.5 | 0.5|

### 2.3 ステップ2：条件付確率の設定

- 以下を調査して、条件付確率を設定します。
    - 本命の相手にチョコをあげる確率
    - 論外の相手にチョコをあげる確率

- これらはの確率、事前確率のときとは違って、沢山の人に聞いて統計データから確率を求めることに意味がある課題と考えられます。
- そしてその結果が以下だったとします。

| タイプ | チョコをあげる確率 | チョコをあげない確率 |
| ---- | ---- | ---- |
|本命 | 0.4 | 0.6 |
|論外 | 0.2 | 0.8 |

- ここまでで、BayesianEstimationAreaChartAnalyzerクラスを使って可視化してみましょう。

In [ ]:
# サンプルデータの定義
prior_probabilities = [0.5, 0.5]
labels = ['本命', '論外']

conditional_probabilities = [
    [0.4, 0.6],
    [0.2, 0.8]
]

# クラスのインスタンス化
analyzer = BayesianEstimationAreaChartAnalyzer(conditional_probabilities, labels, prior_probabilities)

# 1. 事前確率のみを表示するグラフをプロット
analyzer.plot_prior_probabilities_chart(title="事前確率")

# 2. データラベルを2行表示する積み上げ棒グラフをプロット
analyzer.plot_combined_stacked_bar_chart(title="条件付き確率")


### 2.4 ステップ3：観測による事後確率の決定

- 同僚女性からチョコをもらうという事実が発生したとします。
- これも可視化して、事後確率を求めてみましょう。


In [ ]:
# 3. 特定のスタックを強調表示するグラフをプロット
index = 0
analyzer.plot_highlighted_stacked_bar_chart(index, title=f"インデックス{index}を観測した場合")

- 結果は、本命の確率が2/3 = 約66%となりました。

- 逆に、チョコを貰えなかったらどうなるのかも確認しましょう。

In [ ]:
# 3. 特定のスタックを強調表示するグラフをプロット
index = 1
analyzer.plot_highlighted_stacked_bar_chart(index, title=f"インデックス{index}を観測した場合")

- 結果は、本命の確率が約43%となりました。
    - (問題設定のとおりなので間違ってないのですが)チョコがもらえなかっただけで本命の確率が43%あるというのは高い気がします。

### 2.4 主観確率

- 上記で最初に定めた**事前確率**や、求まった**事後確率**は、確率とは言うものの、本当に確率的な(未知の)事象なのではなく、なんらかの結論は出ているけどそれを知らないに過ぎないものになります。
    - 「確率はいくつであると、あなたは思っている」というようなものになります。
    - こうした確率のことを「**主観確率**」と呼びます。
        - ちなみに条件付確率の部分は**客観確率**だったと言えそうです。
        
- ベイズ推定ではこの主観確率が扱えるのが強みといえば強みです。
    - この主観確率というのが怪しいということでベイズ統計は一時期下火だったようです。


----
### 問2 事前確率を色々変えて試してみよう

----

- 上では、事前確率は理由不十分の原理により0.5:0.5としていましたが、ここは別の数値にしてもオッケーです。
    - もともと想像している本命度合いを反映させることができます。

- 以下の3種類の設定で事後確率を計算しましょう。

- 共通部分
    - 条件付確率

| タイプ | チョコをあげる確率 | チョコをあげない確率 |
| ---- | ---- | ---- |
|本命 | 0.4 | 0.6 |
|論外 | 0.2 | 0.8 |



In [64]:
# 共通部分
conditional_probabilities = [
    [0.4, 0.6],
    [0.2, 0.8]
]

#### 問2-1
- 設定1
    - 事前確率 : 
        - 本命 : 0.1
        - 論外 : 0.9
    - 観測結果 : チョコをもらう


In [ ]:
# 設定1
## ここに書く



#### 問2-2

- 設定2
    - 事前確率 : 
        - 本命 : 0.3
        - 論外 : 0.7
    - 観測結果 : チョコをもらう


In [ ]:
# 設定2
## ここに書く


#### 問2-3

- 設定3
    - 事前確率 : 
        - 本命 : 0.8
        - 論外 : 0.2
    - 観測結果 : チョコをもらえない

In [ ]:
# 設定3
## ここに書く


----
## 課題2-1. 事前確率の変化に対する事後確率の変化をグラフにする

----

- Goolge Classroomの課題のファイルに、グラフが書いてあるので、それをまず再現しましょう。
- 描くグラフの仕様
    - 事前確率の本命の率を、0.1から0.9まで、0.1刻みで変化させたものを横軸にする
    - 事後確率の本命の率を縦軸とする
    - プロットするのは折れ線グラフで、以下の3種類とする
        - チョコをもらった場合の事後確率
        - チョコをもらえなかった場合の事後確率
        - 情報がない場合の事後確率(事前確率と同じ)



- そのうえで、条件付確率を変更したバージョンもグラフにしなさい。


- 条件1 : 元のもの

| タイプ | チョコをあげる確率 | チョコをあげない確率 |
| ---- | ---- | ---- |
|本命 | 0.4 | 0.6 |
|論外 | 0.2 | 0.8 |


- 条件2 : 変更したもの

| タイプ | チョコをあげる確率 | チョコをあげない確率 |
| ---- | ---- | ---- |
|本命 | 0.7 | 0.3 |
|論外 | 0.1 | 0.9 |


In [ ]:
# 条件1の場合
## ここに書く


In [ ]:
# 条件2の場合
## ここに書く



----
## 課題2-2. 観測したことで一番確率が変わったのはどこか？

----


- 上記の条件1と条件2で、「情報がない場合」と比べて、観察したことで確率が変動した度合いが大きい場所を見つけたい。
- 「情報がない場合」との他２つのグラフの差分をとって、差が大きいところを見つけよう。
- スライドに数値と、「もらった場合」なのか「もらってない場合」なのかを書き込もう

In [91]:
## 計算に使うコードがあれば書く

## 3. 観測を複数回行う場合

- 参考図書 P.148

- 以下の問題を考えます。
> 眼の前にツボが1つあり、AのツボかBのツボであることはわかっているが、見た目ではどちらかわからない。知識として、Aのツボには9個の白球と1個の黒球が入っており、Bのツボには2個の白球と8個の黒球が入っていることを知っているとする。


- まとめると
    - Aのツボ : [白球:9, 黒球:1]
    - Bのツボ : [白球:2, 黒球:8]

- **IncrementalBayesianAnalyzerクラス**の基本部分を用意しました。これに追記して完成させていきます。
    - 後で完成させるため、まずは**IncrementalBayesianAnalyzer_base**という名前にしています。
    - 上のクラスから最後の計算の部分だけ抜粋して作りました。


In [101]:
class IncrementalBayesianAnalyzer_base:
    def __init__(self, conditional_probabilities, labels, prior_probabilities):
        """
        分析クラスを初期化し、データを保持します。

        Args:
            conditional_probabilities (list of lists): 条件付確率のリスト（形状は [カテゴリ][スタック]）。
            labels (list): 各カテゴリのラベル。
            prior_probabilities (list): 事前確率のリスト（割合指定、合計が1である必要あり）。
        """
        self.conditional_probabilities = np.array(conditional_probabilities)
        self.labels = labels
        self.prior_probabilities = prior_probabilities


    def calculate_posterior_probabilities(self, index):
        """
        指定されたスタックのインデックスに基づいて事後確率を計算し、リストで返します。

        Args:
            index (int): 事後確率を計算するためのスタックのインデックス（0始まり）。

        Returns:
            list: 各カテゴリに対応する事後確率のリスト（合計は1）。
        """
        num_stacks = self.conditional_probabilities.shape[1]
        if index < 0 or index >= num_stacks:
            raise ValueError(f"indexは0から{num_stacks - 1}の範囲で指定してください。")

        # 各カテゴリごとに、事前確率と条件付確率の積を計算
        multiplied_values = []
        for prior, cond_probs in zip(self.prior_probabilities, self.conditional_probabilities):
            cond_prob = cond_probs[index]
            multiplied_value = prior * cond_prob
            multiplied_values.append(multiplied_value)

        # 合計を計算して正規化
        total = sum(multiplied_values)
        if total == 0:
            raise ValueError("合計が0になるため、事後確率を計算できません。")

        posterior_probabilities = [value / total for value in multiplied_values]

        return posterior_probabilities

### 3.1 まずは1個だけ取り出すパターン

- 今、ツボから球を1個だけ取り出して、出てきた球によってツボがAであるかBであるかの確率を求めてみましょう。
- ここでは取り出されたのは白球であるとします。
- 事前確率の設定は、特になんの情報もないとします。

In [ ]:
# 事前確率の設定
prior_probabilities = [0.5, 0.5] # 理由不十分の原理により

# ラベルの設定
labels = ['本命', '論外']

# 条件付確率
conditional_probabilities = [
    [0.9, 0.1],
    [0.2, 0.8]
]

# クラスのインスタンス化
analyzer = IncrementalBayesianAnalyzer_base(conditional_probabilities, labels, prior_probabilities)

# 観測
index = 0 # 白球が取り出された

# 事後確率の計算
posterior_probabilities = analyzer.calculate_posterior_probabilities(index)

# 結果の表示
print(posterior_probabilities)

### 3.2 2回取り出すパターン(その1)
- 今度は、ツボから2回球を取り出してツボAかツボBかを当てることを考えます。
    - 一度球を取り出したら、それは一旦ツボに入れ直すことにします(取り出す際の確率は固定ということ)

- ここでは、上記のクラスを2回使って計算してみましょう。
    - 一度、上と同じように事後確率を計算します。
    - 次に、その**事後確率**を、**事前確率として**再度クラスからオブジェクトを作り、事後確率を計算します。



----
### 問3 ツボの問題で球を2回取り出す(クラスを2回作る)

----

- 上記3.2の設定で事後確率を計算してみましょう。
- 4通りのパターンで計算してみましょう
    - 問3-1 : 白 -> 白
    - 問3-2 : 白 -> 黒
    - 問3-3 : 黒 -> 白
    - 問3-4 : 黒 -> 黒


#### 問3-1 : 白 -> 白

In [ ]:
## ここに書く



#### 問3-2 : 白 -> 黒


In [ ]:
## ここに書く



#### 問3-3 : 黒 -> 白


In [ ]:
## ここに書く



#### 問3-4 : 黒 -> 黒

In [ ]:
## ここに書く

- ここで注目すべきことは、「白 -> 黒」と「黒 -> 白」が同じ結果になることです。
- つまり、このケースでは順番は結果に影響を及ぼさないんですね。


### 3.3 2回取り出すパターン(その2)(クラスを改良する)



- **IncrementalBayesianAnalyzer**クラスを完成させます。



----
### 問4 IncrementalBayesianAnalyzerクラスを作る

----



- 以下の方針で**IncrementalBayesianAnalyzer**クラス作ります。
    - 事後確率(posterior_probabilities)をクラス内に保持する。
        - self.posterior_probabilitiesとして、`__init__`で定義しておく。
        - 最初は`None`を代入しておく。
    - **calculate_posterior_probabilities**メソッドを**update_posterior_probabilities**メソッドに変更
        - このメソッドが呼ばれたら、**現状の事後確率を事前確率に代入する**。
            - ただし、事後確率をまだ一度も求めていない場合は、代入は行わない。
        - あとは**posterior_probabilities**に`self.`をつけるのを忘れずに。


In [120]:
## ここに書く


- 白球 -> 白球 の場合の実行は以下のようになります。

In [ ]:
# 事前確率の設定
prior_probabilities = [0.5, 0.5] # 理由不十分の原理により

# ラベルの設定
labels = ['ツボA', 'ツボB']

# 条件付確率
conditional_probabilities = [
    [0.9, 0.1],
    [0.2, 0.8]
]

# クラスのインスタンス化
analyzer = IncrementalBayesianAnalyzer(conditional_probabilities, labels, prior_probabilities)

# 観測
index = 0 # 白球が取り出された
# 事後確率の計算
posterior_probabilities = analyzer.update_posterior_probabilities(index)

# 結果(途中)の表示
print(posterior_probabilities)

# 観測
index = 0 # 白球が取り出された
# 事後確率の計算
posterior_probabilities = analyzer.update_posterior_probabilities(index)

# 結果(最終)の表示
print(posterior_probabilities)



----
## 課題2-3 ツボから10回球を取り出すときの事後確率の推移をグラフにする。
----

- 3節と同じ問題設定で、ツボから10回球を取り出したとします。

- 2回試行して、以下の順番で球が取り出されたとします。
    - 試行1: [白,黒,黒,白,白,黒,白,白,白,白]
    - 試行2: [黒,白,白,白,白,白,黒,黒,白,白]

- このときの、事後確率の推移を、試行1と試行2の結果を同じグラフに折れ線グラフにまとめなさい。
    - 表示するのはツボAである確率だけでよいです。


In [ ]:
## 課題用のファイルに書いてください